# Transmon CZ gate with tunable couplers

This notebook is similar but not as detail as `examples/fluxonium_multipath_coupling/fluxonium_mpcoupling_6q_x.ipynb`. Please check that notebook for more info.

This example is based on Xu, Y. et al. High-Fidelity, High-Scalability Two-Qubit Gate Scheme for Superconducting Qubits. Phys. Rev. Lett. 125, 240503 (2020). We construct a 3 transmon qubit system, where 2 of them (`q0`, `q1`) are computational qubits and the other one (`q2`) are the coupler. 

The data is read from external json file (in SGM format)

In [17]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import jax.numpy as jnp

from supergrad.helper import Evolve
from supergrad.scgraph import SCGraph
from supergrad.utils.sgm_format import read_sgm_data
from supergrad.utils.gates import cz_gate
from supergrad.utils import compute_fidelity_with_1q_rotation_axis
from supergrad.utils.optimize import scipy_minimize, adam_opt
import pprint


# All data directly reads from the JSON file
# For format, check `supergrad/utils/format_sgm.py`
g: SCGraph = read_sgm_data("transmon_2q1c.json")

# Note this gate natively is CZ^-1
# Target cannot be the original CZ, which is not possible near the working point
target_unitary = np.array([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, -1]])

Note here we have a new argument in the pulse, `amp_type`, which is used to indicate which physical variable the amplitude is on.
This is especially useful when the pulse is not linear to the operator.

In [18]:
pprint.PrettyPrinter(indent=2).pprint(g.nodes["q2"]["pulse"]["p1"])

{ 'amp': Array(2.4, dtype=float64, weak_type=True),
  'amp_type': 'phiej',
  'arguments': {'modulate_wave': False},
  'delay': 0.0,
  'omega_d': Array(0., dtype=float64, weak_type=True),
  'operator_type': 'cosphi_operator',
  'phase': Array(0., dtype=float64, weak_type=True),
  'pulse_type': 'rampcos',
  't_plateau': Array(6.95, dtype=float64, weak_type=True),
  't_ramp': Array(16.44, dtype=float64, weak_type=True)}


In [23]:
# Define the Evolve object
evo = Evolve(g, solver="ode_expm", options={'astep': 2000, 'trotter_order': 2})

# Pick the parameters we want to optimize
# Here we only optimize the control parameters
params = g.convert_graph_to_parameters()
p1 = params["nodes"]["q2"]["pulse"]["p1"]
params_init = {"nodes":
    {
        "q0": {"compensation": params["nodes"]["q0"]["compensation"]},
        "q1": {"compensation": params["nodes"]["q1"]["compensation"]},
        "q2": {"pulse": {"p1": {
            "amp": p1["amp"], "t_ramp": p1["t_ramp"], "t_plateau": p1["t_plateau"]
        }}}}}

# This initial parameters is not as good, takes ~50 cycles to converge. We use a better one to demostrate in fewer cycles.
params_init["nodes"]["q2"]["pulse"]["p1"]["amp"] = jnp.array(2.54)
params_init["nodes"]["q2"]["pulse"]["p1"]["t_plateau"] = jnp.array(7.21)
params_init["nodes"]["q2"]["pulse"]["p1"]["t_ramp"] = jnp.array(16.43)


For optimization, let us use an object function based on the average gate fidelity with leakage. The formula is from Physical Review A 87, 022309 (2013).
$$
	C = 1 - \frac{1}{D(D + 1)}[Tr(PU^{\dagger}_{\text{sim}}P U_{\text{sim}}P)+|Tr(PU^{\dagger}_{\text{sim}} P U_{\text{target}})|^2],   
$$
where $P$ is the projector into the computational subspace $S$, and $D=2^n$ is the dimension of $S$.

In [20]:
def infidelity(params, target_unitary):
    evo.init_quantum_system(params)
    u = evo.final_state(basis="eigen")
    fidelity, res_unitary = compute_fidelity_with_1q_rotation_axis(
        target_unitary, u, compensation_option='no_comp')
    return 1 - fidelity


Show the optimization procedure of control parameters

In [24]:
# Optimize with L-BFGS-B
res = scipy_minimize(infidelity,
                     params_init,
                     args=(target_unitary,),
                     method='l-bfgs-b',
                     logging=True,
                     options={'maxiter': 2000, 'ftol': 1e-5})
print(res)

step: 0
parameters:
{ 'nodes': { 'q0': { 'compensation': { 'post_comp': -0.3714724,
                                       'pre_comp': 0.3714724}},
             'q1': { 'compensation': { 'post_comp': -0.3714724,
                                       'pre_comp': 0.3714724}},
             'q2': { 'pulse': { 'p1': { 'amp': 2.54,
                                        't_plateau': 7.21,
                                        't_ramp': 16.43}}}}}
gradient:
{ 'nodes': { 'q0': { 'compensation': { 'post_comp': 0.20460392735335478,
                                       'pre_comp': 0.2046039273533549}},
             'q1': { 'compensation': { 'post_comp': -0.004956567365926708,
                                       'pre_comp': -0.004956567365926825}},
             'q2': { 'pulse': { 'p1': { 'amp': -4.857841631877335,
                                        't_plateau': 3.2161603169363038,
                                        't_ramp': 6.597323917395401}}}}}
loss: 0.014756
step: 1
paramete